**Neural networks for road segmentation**
========================

Your next task is to train neural network to segment road on images from car cams.

The original dataset is provided by Karlsruhe Institute of Technology (http://www.cvlibs.net/datasets/kitti/eval_road.php). Their images are ±370x1270, but, for simlictiy, we will use 370x370 squares.

One can download the dataset from https://disk.yandex.ru/d/QPOw4hk84-se_w

Here is an example of input data with corresponding ground truth:

In [ ]:
import os
import cv2
import numpy as np

import torchvision.transforms as transforms
from torch.utils.data import Dataset

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def get_paths(path):
    _, _, filenames = next(os.walk(path))

    images_paths = []
    for filename in sorted(filenames):            
        images_paths.append(os.path.join(path, filename))
    
    return np.stack(images_paths)

class RoadDataset(Dataset):
    """Feel free to rewrite it. For ex. cache all images in RAM to increase training speed"""
    def __init__(self, images, masks, transform, aug=None,):
        self.images = sorted(images)
        self.masks = sorted(masks)
        self.transform = transform
        self.aug = aug
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):        
        im_name = self.images[idx]
        mask_name = self.masks[idx]
        
        image = cv2.imread(im_name)[:,:,::-1] / 255.0  # ::-1 to convert from BGR to RGB
        mask = cv2.imread(mask_name, cv2.IMREAD_GRAYSCALE)
        mask = (mask > 0).astype(np.float32)
        
        if self.aug:
            sample = self.aug(
                image=image,
                mask=mask,
            )
        else:
            sample = {
                'image': image,
                'mask': mask,
            }
        
        sample['image'] = self.transform(sample['image']).float()
        sample['mask'] = self.transform(sample['mask']).float()

        return sample

In [ ]:
X_train = get_paths("data/train/images")
y_train = get_paths("data/train/gt")

X_test = get_paths("data/test/images")
y_test = get_paths("data/test/gt")

transform = transforms.Compose([
    transforms.ToTensor(),
])

In [ ]:
train_dataset = RoadDataset(X_train, y_train, transform)
test_dataset = RoadDataset(X_test, y_test, transform)

In [ ]:
sample = train_dataset[4]
image = sample['image']
mask = sample['mask']

plt.subplot(1, 2, 1)
plt.imshow(image.permute(1, 2, 0))
plt.subplot(1, 2, 2)
plt.imshow(mask.permute(1, 2, 0))

Workflow:
---
* Choose correct loss function, write training loop and code for testing. Feel free to use previous HW for inspiration. 
* Train any segmentation neural network from scratch (for example U-Net) and achieve >= 0.75 IoU on test set (40% points). See function to calculate the metric below.
* Use any pretrained model for image classification, convert it for segmentation by adding decoder (don't forget skip-connections) or usign dilated convolutions and achieve >= 0.87 IoU  on test set (60% points).

You're not allowed to do only one thing: train your network on test set.

----
Your final solution will consist of an ipython notebook with code (for final networks training + any experiments with data) and test metric calculation.

Feel free to ask in Telegram chat if something is not clear :3


In [ ]:
print(f"Dataset length {len(train_dataset)}")

Dataset is small so actively use data augmentation: rotations, flip, color-change etc. to prevent overfitting.

Most likely you'll have to pad your images to 512x512 (it divides by 2^5=32, like U-Net wants). Use PadIfNeeded from Albumentations and central crop (see below) after prediction to calculate loss/metrics (you don't want to pay attention on padded values).

----
There is a hard data class imbalance in dataset, so the network output will be biased toward "zero" class. You can either tune the minimal probability threshold for the "road" class, or add class weights in optimized loss. You also can try to use softIoU or DICE loss.

Good luck!

In [ ]:
def calc_iou(prediction, ground_truth):
    n_images = len(prediction)
    intersection, union = 0, 0
    for i in range(n_images):
        intersection += np.logical_and(prediction[i] > 0, ground_truth[i] > 0).astype(np.float32).sum() 
        union += np.logical_or(prediction[i] > 0, ground_truth[i] > 0).astype(np.float32).sum()
    return float(intersection) / union

In [ ]:
def central_crop(images, size):
    current_size = images.size(2)
    border_len = (current_size - size) // 2
    images = images[:, :, border_len:current_size-border_len, border_len:current_size-border_len]

In [ ]:
def train_one_epoch(model, data_loader, optimizer, criterion):
    loss_history = []
    iou_history = []
    
    for x_batch, y_batch in data_loader:
        print(1)
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        print(2)
        output = model(x_batch)
        print(3)
        loss = critetion(output, y_batch)
        print(4)
        optimizer.zero_grad()
        optimizer.step()
        
        loss_history.append(loss.item())
        iou_history.append(calc_iou(output, y_batch))
        
    return loss_history, iou_history

In [ ]:
def eval_model(model, data_loader, criterion):
    loss_history = []
    iou_history = []
    
    for x_batch, y_batch in tqdm(data_loader):
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        
        output = model(x_batch)
        loss = critetion(output, y_batch)
        
        loss_history.append(loss.item())
        iou_history.append(calc_iou(output, y_batch))
        
    return loss_history, iou_history

### Model U-Net

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torchvision.transforms import CenterCrop
from tqdm.notebook import tqdm as tqdm
from torchsummary import summary

In [ ]:
class ConvLayer(nn.Module):
    
    def __init__(self, input_channels, output_channels, 
                 kernel_size=(3, 3), stride=1, dilation=1, use_dropout=True):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=input_channels, 
                               out_channels=output_channels,
                               kernel_size=kernel_size,
                               stride=stride,
                               dilation=dilation)
        self.conv2 = nn.Conv2d(in_channels=output_channels, 
                               out_channels=output_channels,
                               kernel_size=kernel_size,
                               stride=stride,
                               dilation=dilation)
        self.dropout2d = nn.Dropout2d(0.2)
        
        self.use_dropout = use_dropout
        
    
    def forward(self, input):
        x = self.conv1(input)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        
        if self.use_dropout:
            x = self.dropout2d(x)
        
        return x

In [ ]:
class DownScaller(nn.Module):
    
    def __init__(self, input_channels, output_channels,
                 kernel_size=(3, 3), stride=1, dilation=1, use_dropout=True):
        super().__init__()
        self.maxpool = nn.MaxPool2d(kernel_size = (2, 2))
        self.conv_layer = ConvLayer(input_channels, output_channels, kernel_size,
                                    stride, dilation, use_dropout)
    
    
    def forward(self, input):
        x = self.maxpool(input)
        x = self.conv_layer(x)

        return x

In [ ]:
class UpScaller(nn.Module):
    
    def __init__(self, input_channels, output_channels,
               kernel_size=(2, 2), stride=2, dilation=1, use_dropout=True):
        super().__init__()
        self.conv_layer = ConvLayer(input_channels, output_channels,
                                    kernel_size=(3, 3), stride=1, dilation=1)
        self.conv_transposed = nn.ConvTranspose2d(output_channels, output_channels // 2,
                                          kernel_size=kernel_size, stride=stride)
    
    
    def forward(self, input):
        x = self.conv_layer(input)
        x = self.conv_transposed(x)
        
        return x
        

In [ ]:
class UNet(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        # Input
        self.conv1 = ConvLayer(3, 64)
        
        # Downscalling
        self.down1 = DownScaller(64, 128)
        self.down2 = DownScaller(128, 256)
        self.down3 = DownScaller(256, 512)
        
        # Bridge
        self.maxpool = nn.MaxPool2d(kernel_size=(2, 2))   
        self.up1 = UpScaller(512, 1024)
        
        # Upscalling
        self.up2 = UpScaller(1024, 512)
        self.up3 = UpScaller(512, 256)
        self.up4 = UpScaller(256, 128)
        
        # Output
        self.conv2 = ConvLayer(128, 64)
        self.conv3 = nn.Conv2d(64, 1, kernel_size=(3, 3), stride=1, dilation=1)
        
    
    
    def forward(self, input):
        x_1 = self.conv1(input)
        
        x_2 = self.down1(x_1)
        x_3 = self.down2(x_2)
        x_4 = self.down3(x_3)
        
        x_5 = self.maxpool(x_4)
        x_6 = self.up1(x_5)
        
        x_4_skip = self.center_crop(x_4, x_6)
        x_7 = self.up2(x_4_skip)
        
        x_3_skip = self.center_crop(x_3, x_7)
        x_8 = self.up3(x_3_skip)
        
        x_2_skip = self.center_crop(x_2, x_8)
        x_9 = self.up4(x_2_skip)
        
        x_1_skip = self.center_crop(x_1, x_9)
        x_10 = self.conv2(x_1_skip)
        
        output = self.conv3(x_10)
        
        return output
    
    def center_crop(self, source_tensor, target_tensor):
        (_, _, h, w) = target_tensor.shape
        crop = CenterCrop((h, w))(source_tensor)
        crop_concat = torch.cat([crop, target_tensor], dim=1)
        
        return crop_concat
        

In [ ]:
# CHECK
unet = UNet()
a = torch.rand([4, 3, 572, 572])
out = unet(a)

print(out.shape)

### Train U-Net

In [ ]:
device = torch.device('cuda')

model = UNet().to(device)
summary(model, input_size=train_dataset[0]['image'].shape)


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCEWithLogitsLoss().to(device)

In [ ]:
BATCH_SIZE = 1
EPOCHS = 2

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, shuffle=False, batch_size=BATCH_SIZE,
                                           num_workers=12, persistent_workers=True, pin_memory=True)
test_loader = torch.utils.data.DataLoader(test_dataset, shuffle=False, batch_size=BATCH_SIZE, 
                                          num_workers=12, persistent_workers=True, pin_memory=True)

In [ ]:
torch.cuda.empty_cache()

for epoch in tqdm(range(EPOCHS), desc='EPOCH'):
    
    model.train(True)
    loss_train = []
    iou_train = []

    for batch in tqdm(train_loader, total=len(train_dataset)):
        x_batch, y_batch = batch['image'].to(device), batch['mask'].to(device)

        output = model(x_batch)

        loss = criterion(output, y_batch.squeeze(1))

        optimizer.zero_grad()
        optimizer.step()

        loss_train.append(loss.item())
        iou_train.append(calc_iou(output, y_batch))
    
#     model.train(False)
#     loss_test, iou_test = eval_model(model, test_loader, criterion)
    
    print(f'EPOCH {epoch + 1} / {EPOCHS}\n'
          f'Train loss: {torch.mean(loss_train):.3f}  iou: {torch.mean(iou_train):.3f}'
#           f'Test loss: {torch.mean(loss_test):.3f}  iou: {torch.mean(iou_test):.3f}'
         )